In [ ]:
# Architecture Single Discriminator
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        return residual + out

class Generator(nn.Module):
    def __init__(self, num_res_blocks=5, num_channels=64):
        super(Generator, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, num_channels, kernel_size=9, stride=1, padding=4),
            nn.PReLU()
        )
        self.res_blocks = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_res_blocks)])
        self.middle = nn.Sequential(
            nn.Conv2d(num_channels, num_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_channels)
        )
        self.upsample_blocks = nn.Sequential(
            nn.Conv2d(num_channels, num_channels * 4, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.PReLU(),
            nn.Conv2d(num_channels, num_channels * 4, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.PReLU()
        )
        self.final = nn.Conv2d(num_channels, 3, kernel_size=9, stride=1, padding=4)

    def forward(self, x):
        x = self.initial(x)
        residual = x
        x = self.res_blocks(x)
        x = self.middle(x) + residual
        x = self.upsample_blocks(x)
        x = self.final(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        features = [64, 128, 256, 512]
        layers = []
        for i in range(1, len(features)):
            layers.append(nn.Conv2d(features[i-1], features[i], kernel_size=3, stride=1, padding=1))
            layers.append(nn.BatchNorm2d(features[i]))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            layers.append(nn.Conv2d(features[i], features[i], kernel_size=3, stride=2, padding=1))
            layers.append(nn.BatchNorm2d(features[i]))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
        self.middle = nn.Sequential(*layers)
        self.final = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(1024, 1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.middle(x)
        x = self.final(x)
        print(x.shape)
        return x


In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
print(device)

cpu


In [ ]:
#4
!nvidia-smi

In [ ]:
from datasets import load_dataset

div2k_dataset = load_dataset("eugenesiow/Div2k")
print(div2k_dataset)

train_data = div2k_dataset["train"]
validation_data = div2k_dataset["validation"]

for sample in train_data[:5]:
    print(sample)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
from datasets import load_dataset

class Div2kDataset(Dataset):
    def __init__(self, div2k_data, transform=None, target_size=(256, 256)):
        self.div2k_data = div2k_data
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.div2k_data)

    def __getitem__(self, idx):
        lr_image_path = self.div2k_data[idx]['lr']
        hr_image_path = self.div2k_data[idx]['hr']
        lr_image = Image.open(lr_image_path).convert('RGB')
        hr_image = Image.open(hr_image_path).convert('RGB')
        lr_image = lr_image.resize(self.target_size, Image.BICUBIC)
        hr_image = hr_image.resize(self.target_size, Image.BICUBIC)
        if self.transform:
            lr_image = self.transform(lr_image)
            hr_image = self.transform(hr_image)
        return lr_image, hr_image

div2k_dataset = load_dataset("eugenesiow/Div2k", split='train[:200]')
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = Div2kDataset(div2k_dataset, transform=transform)
BATCH_SIZE = 1
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
from datasets import load_dataset

class Div2kDataset(Dataset):
    def __init__(self, div2k_data, transform=None):
        self.div2k_data = div2k_data
        self.transform = transform  

    def __len__(self):
        return len(self.div2k_data)

    def __getitem__(self, idx):
        lr_image_path = self.div2k_data[idx]['lr']
        hr_image_path = self.div2k_data[idx]['hr']
        lr_image = Image.open(lr_image_path).convert('RGB')
        hr_image = Image.open(hr_image_path).convert('RGB')
        low_resolution_transform = transforms.Compose([
            transforms.Resize((96, 96), Image.BICUBIC),
            transforms.ToTensor(),
        ])
        high_resolution_transform = transforms.Compose([
            transforms.Resize((384, 384), Image.BICUBIC),
            transforms.ToTensor(),
        ])
        lr_image = low_resolution_transform(lr_image)
        hr_image = high_resolution_transform(hr_image)
        return lr_image, hr_image

div2k_dataset = load_dataset("eugenesiow/Div2k", split='train[:200]')
train_dataset = Div2kDataset(div2k_dataset)
BATCH_SIZE = 1
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
lr_example, hr_example = next(iter(train_loader))
lr_dimensions = tuple(lr_example.shape[-2:])
hr_dimensions = tuple(hr_example.shape[-2:])
print("LR image dimensions:", lr_dimensions)
print("HR image dimensions:", hr_dimensions)


In [ ]:
import torchvision.models as models

class VGGFeatureExtractor(nn.Module):
    def __init__(self, feature_layer=34, use_bn=False):
        super(VGGFeatureExtractor, self).__init__()
        vgg = models.vgg19(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(vgg.features.children())[:feature_layer+1])
        self.use_bn = use_bn

    def forward(self, x):
        if self.use_bn:
            mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(x.device)
            std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(x.device)
            x = (x - mean) / std
        return self.feature_extractor(x)

feature_extractor = VGGFeatureExtractor(feature_layer=34, use_bn=True).to(device)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

checkpoint_path = r'C:\Users\91962\Downloads\projcv\model_checkpoint3.pth'

generator = Generator()
discriminator = Discriminator()
generator.to(device)
discriminator.to(device)

criterion_bce = nn.BCEWithLogitsLoss()
criterion_pixel = nn.L1Loss()
criterion_feature = nn.MSELoss()

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 100
adversarial_loss_weight = 1e-3
pixel_loss_weight = 1
log_interval = 100
save_interval = 2

for epoch in range(num_epochs):
    generator.train()
    discriminator.train()
    for batch_idx, (blurred_images, sharp_images) in enumerate(train_loader):
        blurred_images = blurred_images.to(device)
        sharp_images = sharp_images.to(device)
        optimizer_D.zero_grad()
        fake_sharp_images = generator(blurred_images)
        real_outputs = discriminator(sharp_images)
        fake_outputs = discriminator(fake_sharp_images.detach())
        d_loss = (criterion_bce(real_outputs, torch.ones_like(real_outputs)) + criterion_bce(fake_outputs, torch.zeros_like(fake_outputs)))/2
        d_loss.backward()
        optimizer_D.step()
        optimizer_G.zero_grad()
        fake_sharp_images1 = discriminator(fake_sharp_images)
        adversarial_loss = criterion_bce(fake_sharp_images1, torch.ones_like(fake_sharp_images1))
        sharp_images_resized = F.interpolate(sharp_images, size=fake_sharp_images1.shape[2:], mode='nearest')
        pixel_loss = criterion_pixel(fake_sharp_images1, sharp_images_resized)
        with torch.no_grad():
            real_features = feature_extractor(sharp_images)
            fake_features = feature_extractor(fake_sharp_images1)
            real_features_resized = F.interpolate(real_features, size=fake_features.shape[2:], mode='nearest')
        feature_loss = criterion_feature(fake_features, real_features_resized)
        total_loss = (adversarial_loss_weight * adversarial_loss + pixel_loss_weight * pixel_loss)/2
        total_loss.backward()
        optimizer_G.step()
        if batch_idx % log_interval == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], "
                  f"Generator Loss: {total_loss.item():.4f}, Discriminator Loss: {d_loss.item():.4f}")
    if (epoch + 1) % save_interval == 0:
        torch.save({
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'optimizer_G_state_dict': optimizer_G.state_dict(),
            'optimizer_D_state_dict': optimizer_D.state_dict(),
            'total_loss': total_loss
        }, checkpoint_path)


In [ ]:
# Training 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

checkpoint_path = r'C:\Users\91962\Downloads\projcv\model_checkpoint2.pth'

generator = Generator().to(device)
discriminator = Discriminator().to(device)

criterion_bce = nn.BCEWithLogitsLoss()
criterion_pixel = nn.L1Loss()
criterion_feature = nn.MSELoss()

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 100
adversarial_loss_weight = 1e-3
pixel_loss_weight = 1
feature_loss_weight = 0.01  
log_interval = 100
save_interval = 2

for epoch in range(num_epochs):
    generator.train()
    discriminator.train()
    for batch_idx, (low_res_images, high_res_images) in enumerate(train_loader):
        low_res_images, high_res_images = low_res_images.to(device), high_res_images.to(device)
        
        optimizer_D.zero_grad()
        fake_high_res_images = generator(low_res_images)
        real_output = discriminator(high_res_images)
        fake_output = discriminator(fake_high_res_images.detach())
        loss_real = criterion_bce(real_output, torch.ones_like(real_output))
        loss_fake = criterion_bce(fake_output, torch.zeros_like(fake_output))
        d_loss = (loss_real + loss_fake) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        fake_output = discriminator(fake_high_res_images)
        g_loss_fake = criterion_bce(fake_output, torch.ones_like(fake_output))
        pixel_loss = criterion_pixel(fake_high_res_images, high_res_images)

        real_features = feature_extractor(high_res_images)
        fake_features = feature_extractor(fake_high_res_images)
        feature_loss = criterion_feature(fake_features, real_features)

        g_loss = g_loss_fake + pixel_loss_weight * pixel_loss + feature_loss_weight * feature_loss
        g_loss.backward()
        optimizer_G.step()

        if batch_idx % log_interval == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    if (epoch + 1) % save_interval == 0:
        torch.save({
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'optimizer_G_state_dict': optimizer_G.state_dict(),
            'optimizer_D_state_dict': optimizer_D.state_dict(),
        }, checkpoint_path)


In [14]:
from PIL import Image
from torchvision import transforms

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    if transform:
        image = transform(image)
    image = image.unsqueeze(0)  
    return image


In [ ]:
import torch
from torchvision import transforms
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
from PIL import Image

checkpoint = torch.load(r'C:\Users\91962\Downloads\projcv\model_checkpoint6.pth', map_location='cpu')
generator = Generator()
generator.load_state_dict(checkpoint['generator_state_dict'])
generator.eval()

image_path = 'C:/Users/91962/Downloads/archive (4)/train/FAKE/1000 (4).jpg'  

transform = transforms.Compose([
    transforms.Resize((96, 96)),  
    transforms.ToTensor()         
])

lr_image = Image.open(image_path).convert('RGB')
lr_image = transform(lr_image)
lr_image = lr_image.unsqueeze(0) 

device = torch.device('cpu')  
lr_image = lr_image.to(device)


with torch.no_grad():
    hr_image = generator(lr_image)  

hr_image = hr_image.squeeze(0)  
hr_image = torch.clamp(hr_image, 0, 1)  
to_pil = ToPILImage()
hr_image_pil = to_pil(hr_image.cpu())  


plt.imshow(hr_image_pil)
plt.axis('off')  
plt.show()




In [ ]:
#Evaluation for single discriminator architecture
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize
from datasets import load_dataset

class Div2kValidationDataset(Dataset):
    def __init__(self, div2k_data, transform=None, target_size=(384, 384)):
        self.div2k_data = div2k_data
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.div2k_data)

    def __getitem__(self, idx):
        lr_image_path = self.div2k_data[idx]['lr']
        hr_image_path = self.div2k_data[idx]['hr']
        lr_image = Image.open(lr_image_path).convert('RGB')
        hr_image = Image.open(hr_image_path).convert('RGB')
        
        hr_image = hr_image.resize(self.target_size, Image.BICUBIC)
        
        if self.transform:
            lr_image = self.transform(lr_image)
        
        hr_image = transforms.ToTensor()(hr_image)
        
        return lr_image, hr_image

div2k_validation_dataset = load_dataset("eugenesiow/Div2k", split='validation[:100]')
transform = transforms.Compose([
    transforms.Resize((96, 96)), 
    transforms.ToTensor()
])
validation_dataset = Div2kValidationDataset(div2k_validation_dataset, transform=transform, target_size=(384, 384))
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)

device = torch.device('cpu')  
checkpoint = torch.load(r'C:\Users\91962\Downloads\model_checkpoint2.pth', map_location=device)
generator = Generator().to(device)
generator.load_state_dict(checkpoint['generator_state_dict'])
generator.eval()

psnrs = []
ssims = []
for lr_image, hr_image_true in validation_loader:
    lr_image = lr_image.to(device)
    hr_image_true = hr_image_true.to(device)  
    with torch.no_grad():
        hr_image_pred = generator(lr_image)
        hr_image_pred = hr_image_pred.clamp(0, 1).permute(0, 2, 3, 1).cpu().numpy()
        hr_image_true = hr_image_true.permute(0, 2, 3, 1).cpu().numpy()

    hr_image_pred = (hr_image_pred * 255).astype(np.uint8)
    hr_image_true = (hr_image_true * 255).astype(np.uint8)

    for true, pred in zip(hr_image_true, hr_image_pred):
        psnr = peak_signal_noise_ratio(true, pred, data_range=255)
        ssim = structural_similarity(true, pred, multichannel=True, data_range=255)
        psnrs.append(psnr)
        ssims.append(ssim)

print(f"PSNR: {np.max(psnrs):.2f}")
print(f"SSIM: {np.max(ssims):.4f}")


In [4]:
# Architecture for Double discriminator
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        return residual + out

class Generator(nn.Module):
    def __init__(self, num_res_blocks=8, num_channels=64):
        super(Generator, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, num_channels, kernel_size=9, stride=1, padding=4),
            nn.PReLU()
        )
        self.res_blocks = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_res_blocks)])
        self.middle = nn.Sequential(
            nn.Conv2d(num_channels, num_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_channels)
        )
        self.upsample_blocks = nn.Sequential(
            nn.Conv2d(num_channels, num_channels * 4, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.PReLU(),
            nn.Conv2d(num_channels, num_channels * 4, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.PReLU()
        )
        self.final = nn.Conv2d(num_channels, 3, kernel_size=9, stride=1, padding=4)

    def forward(self, x):
        x = self.initial(x)
        residual = x
        x = self.res_blocks(x)
        x = self.middle(x) + residual
        x = self.upsample_blocks(x)
        x = self.final(x)
        return x

class DiscriminatorGlobal(nn.Module):
    def __init__(self):
        super(DiscriminatorGlobal, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.final = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.final(x)
        return x

class DiscriminatorLocal(nn.Module):
    def __init__(self):
        super(DiscriminatorLocal, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.final = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.final(x)
        return x


In [ ]:
#Evaluation for double discriminator architecture

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize
from datasets import load_dataset

class Div2kValidationDataset(Dataset):
    def __init__(self, div2k_data, transform=None, target_size=(384, 384)):
        self.div2k_data = div2k_data
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.div2k_data)

    def __getitem__(self, idx):
        lr_image_path = self.div2k_data[idx]['lr']
        hr_image_path = self.div2k_data[idx]['hr']
        lr_image = Image.open(lr_image_path).convert('RGB')
        hr_image = Image.open(hr_image_path).convert('RGB')
        
        hr_image = hr_image.resize(self.target_size, Image.BICUBIC)
        
        if self.transform:
            lr_image = self.transform(lr_image)
        
        hr_image = transforms.ToTensor()(hr_image)
        
        return lr_image, hr_image

div2k_validation_dataset = load_dataset("eugenesiow/Div2k", split='validation[:100]')
transform = transforms.Compose([
    transforms.Resize((96, 96)),  
    transforms.ToTensor()
])
validation_dataset = Div2kValidationDataset(div2k_validation_dataset, transform=transform, target_size=(384, 384))
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)


device = torch.device('cpu')  
checkpoint = torch.load(r'C:\Users\91962\Downloads\model_checkpoint3 (2).pth', map_location=device)
generator = Generator().to(device)
generator.load_state_dict(checkpoint['generator_state_dict'])
generator.eval()

psnrs = []
ssims = []
for lr_image, hr_image_true in validation_loader:
    lr_image = lr_image.to(device)
    hr_image_true = hr_image_true.to(device)  
    
    with torch.no_grad():
        hr_image_pred = generator(lr_image)
        hr_image_pred = hr_image_pred.clamp(0, 1).permute(0, 2, 3, 1).cpu().numpy()
        hr_image_true = hr_image_true.permute(0, 2, 3, 1).cpu().numpy()

    hr_image_pred = (hr_image_pred * 255).astype(np.uint8)
    hr_image_true = (hr_image_true * 255).astype(np.uint8)

    for true, pred in zip(hr_image_true, hr_image_pred):
        psnr = peak_signal_noise_ratio(true, pred, data_range=255)
        ssim = structural_similarity(true, pred, multichannel=True, data_range=255)
        psnrs.append(psnr)
        ssims.append(ssim)

print(f"PSNR: {np.max(psnrs):.2f}")
print(f"SSIM: {np.max(ssims):.4f}")
